# Construire des graphiques avec Python

Lino Galiana  
2025-06-14

<div class="badge-container"><div class="badge-text">Pour essayer les exemples présents dans ce tutoriel :</div><a href="https://github.com/linogaliana/python-datascientist-notebooks/blob/main/notebooks/visualisation/matplotlib.ipynb" target="_blank" rel="noopener"><img src="https://img.shields.io/static/v1?logo=github&label=&message=View%20on%20GitHub&color=181717" alt="View on GitHub"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/vscode-python?autoLaunch=true&name=«matplotlib»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-vscode.sh»&init.personalInitArgs=«visualisation%20matplotlib%20correction»" target="_blank" rel="noopener"><img src="https://custom-icon-badges.demolab.com/badge/SSP%20Cloud-Lancer_avec_VSCode-blue?logo=vsc&logoColor=white" alt="Onyxia"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/jupyter-python?autoLaunch=true&name=«matplotlib»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-jupyter.sh»&init.personalInitArgs=«visualisation%20matplotlib%20correction»" target="_blank" rel="noopener"><img src="https://img.shields.io/badge/SSP%20Cloud-Lancer_avec_Jupyter-orange?logo=Jupyter&logoColor=orange" alt="Onyxia"></a>
<a href="https://colab.research.google.com/github/linogaliana/python-datascientist-notebooks-colab//blob/main//notebooks/visualisation/matplotlib.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a><br></div>

La pratique de la *data visualisation* se fera, dans ce cours, en répliquant des graphiques qu’on peut trouver sur
la page de l’*open data* de la ville de Paris
[ici](https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/dataviz/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name) ou en proposant des alternatives à ceux-ci sur les mêmes données.

L’objectif de ce chapitre n’est pas de faire un inventaire complet des graphiques pouvant être fait avec `Python`, ce serait long, assez insipide et peu pertinent car des sites le font déjà très bien à partir d’une grande variété d’exemple, notamment le site [python-graph-gallery.com/](https://python-graph-gallery.com/). L’objectif est plutôt d’illustrer, par la pratique, quelques enjeux liés à l’utilisation des principales librairies graphiques de `Python`.

On peut distinguer quelques grandes familles de représentations graphiques: les représentations de distributions propres à une variable, les représentations de relations entre plusieurs variables, les cartes qui permettent de représenter dans l’espace une ou plusieurs variables…

Ces familles se ramifient elles-mêmes en de multiples types de figures. Par exemple, selon la nature du phénomène, les représentations de relations peuvent prendre la forme d’une série temporelle (évolution d’une variable dans le temps), d’un nuage de point (corrélation entre deux variables), d’un diagramme en barre (pour souligner le rapport relatif entre les valeurs d’une variable en fonction d’une autre), etc.

Plutôt qu’un inventaire à la Prévert des types de visualisations possibles, ce chapitre et le suivant vont plutôt proposer quelques visualisations qui pourraient donner envie d’aller plus loin dans l’analyse avant la mise en oeuvre d’une forme de modélisation. Ce chapitre est consacré aux visualisations traditionnelles, le [suivant](../../content/visualisation/maps.qmd) est dédié à la cartographie. Ces deux chapitres font partie d’un tout visant à offrir les premiers éléments pour synthétiser l’information présente dans un jeu de données.

Le pas suivant est d’approfondir le travail de communication et de synthèse par le biais de communications pouvant prendre des formes aussi diverses que des rapports, des publications scientifiques ou articles, des présentations, une application interactive, un site web ou des *notebooks* comme ceux proposés par ce cours. Le principe général est identique quelle que soit le *medium* utilisé et intéresse particulièrement les *data scientists* lorsqu’ils font appel à de l’exploitation intensive de données. Ce sera l’objet d’un chapitre futur de ce cours[1].

# 1. Données

Ce chapitre s’appuie sur les données de comptage des passages de vélo dans les points de mesure parisiens diffusés sur le site de l’*open data* de la ville de Paris.

L’exploitation de l’historique récent a été grandement facilité par la diffusion des données au format `Parquet`, un format moderne plus pratique que le CSV. Pour en savoir plus sur ce format, vous pouvez consulter les ressources évoquées dans le paragraphe consacré à ce format dans le [chapitre d’approfondissement](../../content/manipulation/02_pandas_suite.qmd).

[1] This chapter will be built around the [`Quarto`](https://quarto.org/) ecosystem. In the meantime, you can consult the excellent documentation of this ecosystem and practice, which is the best way to learn.

In [7]:
import os
import requests
from tqdm import tqdm
import pandas as pd
import duckdb

url = "https://minio.lab.sspcloud.fr/lgaliana/data/python-ENSAE/comptage-velo-donnees-compteurs.parquet"
# problem with https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptage-velo-donnees-compteurs/exports/parquet?lang=fr&timezone=Europe%2FParis

filename = 'comptage_velo_donnees_compteurs.parquet'


# DOWNLOAD FILE --------------------------------

# Perform the HTTP request and stream the download
response = requests.get(url, stream=True)

if not os.path.exists(filename):
    # Perform the HTTP request and stream the download
    response = requests.get(url, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Get the total size of the file from the headers
        total_size = int(response.headers.get('content-length', 0))

        # Open the file in write-binary mode and use tqdm to show progress
        with open(filename, 'wb') as file, tqdm(
                desc=filename,
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
        ) as bar:
            # Write the file in chunks
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:  # filter out keep-alive chunks
                    file.write(chunk)
                    bar.update(len(chunk))
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
else:
    print(f"The file '{filename}' already exists.")

# READ FILE AND CONVERT TO PANDAS --------------------------

query = """
SELECT id_compteur, nom_compteur, id, sum_counts, date
FROM read_parquet('comptage_velo_donnees_compteurs.parquet')
"""

# READ WITH DUCKDB AND CONVERT TO PANDAS
df = duckdb.sql(query).df()

df.head(3)

# 2. Premières productions graphiques avec l’API `Matplotlib` de `Pandas`

Chercher à produire une visualisation parfaite du premier coup est illusoire. Il est beaucoup plus réaliste d’améliorer graduellement une représentation graphique afin, petit à petit, de mettre en avant les effets de structure dans un jeu de données.

Nous allons donc commencer par nous représenter la distribution des passages aux principales stations de mesure. Pour cela nous allons produire rapidement un *barplot* puis l’améliorer graduellement.

Dans cette partie, nous allons ainsi reproduire les deux premiers graphiques de la [page d’analyse des données](https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/dataviz/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name) : *Les 10 compteurs avec la moyenne horaire la plus élevée* et *Les 10 compteurs ayant comptabilisé le plus de vélos*. Les valeurs chiffrées des graphiques peuvent être différentes de celles de la page en ligne, c’est normal, car nous ne travaillons pas systématiquement sur les données ayant la même fraîcheur que celles en ligne.

Pour importer les librairies graphiques que nous utiliserons dans ce chapitre, il faut faire

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *

## 2.1 Comprendre, en quelques mots, le principe de `matplotlib`

`matplotlib` date du début des années 2000 et a émergé pour proposer une alternative en `Python` à la création de graphiques sous `Matlab`, un logiciel propriétaire de calcul numérique. `matplotlib` est donc une librairie assez ancienne, antérieure à l’émergence de `Python` dans l’écosystème du traitement de données. Cela s’en ressent sur la logique de construction de `matplotlib` qui n’est pas toujours intuitive lorsqu’on est familier de l’écosystème moderne de la *data science*. Heureusement, il existe de nombreuses librairies qui s’appuient sur `matplotlib` mais qui visent à fournir une syntaxe plus familière aux *data scientists*.

`matplotlib` propose principalement deux niveaux d’abstraction: la figure et les axes. La figure est, en quelque sorte, la “toile” globale qui contient un ou plusieurs axes dans lesquels s’inséreront des graphiques. Selon les cas, il faudra jouer avec les paramètres de figure ou d’axe, ce qui rend très flexible la construction d’un graphique mais peut également être déroutant car on ne sait jamais trop quel niveau d’abstraction il faut modifier pour mettre à jour sa figure[1]. Comme le montre la <a href="#fig-matplotlib" class="quarto-xref">Figure 2.1</a>, tous les éléments d’une figure sont paramétrables.

<figure id="fig-matplotlib">
<img src="https://matplotlib.org/stable/_images/anatomy.png" />
<figcaption>Figure 2.1: Comprendre l’architecture d’une figure <code>matplotlib</code> (Source: <a href="https://matplotlib.org/stable/users/explain/quick_start.html">documentation officielle</a>)</figcaption>
</figure>

En pratique, il existe deux manières de créer et mettre à jour sa figure selon qu’on préfère passer par:

-   l’approche explicite, héritière d’une logique de programmation orientée objet, où on crée des objets `Figure` et `Axes` et met à jour ceux-ci.
-   l’approche implicite, basée sur l’interface `pyplot` qui utilise une succession de fonctions pour mettre à jour les objets créés implicitement.

## Approche explicite (approche orientée POO)

``` python
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 2, 100)  # Sample data.

# Note that even in the OO-style, we use `.pyplot.figure` to create the Figure.
fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')
ax.plot(x, x, label='linear')  # Plot some data on the Axes.
ax.plot(x, x**2, label='quadratic')  # Plot more data on the Axes...
ax.plot(x, x**3, label='cubic')  # ... and some more.
ax.set_xlabel('x label')  # Add an x-label to the Axes.
ax.set_ylabel('y label')  # Add a y-label to the Axes.
ax.set_title("Simple Plot")  # Add a title to the Axes.
ax.legend()  # Add a legend.
```

Source: [Documentation officielle de `matplotlib`](https://matplotlib.org/stable/users/explain/quick_start.html)

## Approche implicite

``` python
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 2, 100)  # Sample data.

plt.figure(figsize=(5, 2.7), layout='constrained')
plt.plot(x, x, label='linear')  # Plot some data on the (implicit) Axes.
plt.plot(x, x**2, label='quadratic')  # etc.
plt.plot(x, x**3, label='cubic')
plt.xlabel('x label')
plt.ylabel('y label')
plt.title("Simple Plot")
plt.legend()
```

Source: [Documentation officielle de `matplotlib`](https://matplotlib.org/stable/users/explain/quick_start.html)

Ces éléments constituent le minimum pour comprendre la logique de `matplotlib`. Pour être plus à l’aise avec ces concepts, la pratique répétée est indispensable.

## 2.2 Découvrir `matplotlib` par l’intermédiaire de `Pandas`

Les 10 principales stations à l’issue de la question 1 représentent celles ayant la moyenne la plus élevée pour le volume de passages de vélos. Ces données réordonnées permettent de créer un graphique lisible et de mettre en avant les stations les plus fréquentées.

[1] Thankfully, with a vast amount of online code using `matplotlib`, code assistants like `ChatGPT` or `Github Copilot` are invaluable for creating charts based on instructions.

Figure 1, sans travail sur le style, présente les données sous forme de *barplot* basique. Bien qu’elle montre les informations essentielles, elle manque de mise en page esthétique, de couleurs harmonieuses et d’annotations claires, nécessaires pour améliorer la lisibilité et l’impact visuel.

Figure 2 sans travail sur le style:

On commence à avoir quelque chose qui commence à transmettre un message synthétique sur la nature des données. On peut néanmoins remarquer plusieurs éléments problématiques (par exemple les labels) mais aussi des éléments ne correspondant pas (les titres des axes, etc.) ou manquants (le nom du graphique…).

Comme les graphiques produits par `Pandas` suivent la logique très flexible de `matplotlib`, il est possible de les customiser. Cependant, cela demande généralement beaucoup de travail et la grammaire `matplotlib` n’est pas aussi normalisée que celle de `ggplot` en `R`. Si on désire rester dans l’écosystème `matplotlib`, il est préférable de directement utiliser `seaborn`, qui offre quelques arguments prêts à l’emploi. Sinon on peut basculer sur l’écosystème `plotnine` qui offrira une syntaxe normalisée pour modifier les différents

# 3. Utiliser directement `seaborn`

## 3.1 Comprendre `seaborn` en quelques lignes

`seaborn` est une interface haut niveau au dessus de `matplotlib`. Ce package offre un ensemble de fonctionnalités pour créer des figures ou des axes `matplotlib` directement depuis une fonction admettant de nombreux arguments et, si besoin d’aller plus loin dans la customisation, d’utiliser les fonctionnalités de `matplotlib` pour mettre à jour la figure, que ce soit par le biais de l’approche implicite ou explicite décrites précédemment.

Comme pour `matplotlib`, `seaborn` permet de faire la même figure de multiples manières. `seaborn` hérite de la dualité axes-figures de `matplotlib` et il faudra souvent jouer avec un niveau ou l’autre. La principale caractéristique de `seaborn` est d’offrir quelques points d’entrée standardisés, par exemple `seaborn.relplot` ou `seaborn.catplot`, et une logique d’*inputs* basée sur le `DataFrame` là où `matplotlib` est structurée autour du *array* `Numpy`.

La figure comporte maintenant un message mais il est encore peu lisible. Il y a plusieurs manières de faire un *barplot* en `seaborn`. Les deux principales sont :

-   `sns.catplot` ;
-   `sns.barplot`.

On propose d’utiliser `sns.catplot` pour cet exercice. Il s’agit d’un point d’entrée assez fréquent pour faire des graphiques d’une variable discrétisée.

## 3.2 Le diagramme en barre (*barplot*)

À l’issue de la question 2, c’est-à-dire en utilisant `seaborn` pour reproduire de manière minimale un *barplot*, on obtient :

Après quelques réglages esthétiques, à l’issue des questions 3 et 4, on obtient une figure proche de celle du portail *open data* parisien.

Les paramètres supplémentaires proposés à la question 4 permettent finalement d’obtenir la figure

On comprend ainsi que le boulevard de Sébastopol est le plus emprunté, ce qui ne vous surprendra pas si vous faites du vélo à Paris. Néanmoins, si vous n’êtes pas familiers avec la géographie parisienne, cela sera peu informatif pour vous, vous allez avoir besoin d’une représentation graphique supplémentaire : une carte ! Nous verrons ceci lors d’un prochain chapitre.

## 3.3 Un exemple d’alternative au *barplot*, le *lollipop chart*

Les diagrammes en bâtons (*barplot*) sont extrêmement communs, sans doute à cause de l’héritage d’Excel où ces graphiques sont faisables en deux clics. Néanmoins, en ce qui concerne le message à transmettre, ils sont loin d’être parfaits. Par exemple, les barres prennent beaucoup d’espace visuel, ce qui peut brouiller le message à transmettre sur le rapport entre les observations.

Sur le plan sémiologique, c’est-à-dire sur le plan de l’efficacité du message à transmettre, les *lollipop charts* sont préférables : ils transmettent la même information mais avec moins de signes visuels pouvant brouiller sa compréhension.

Les *lollipop charts* ne sont pas parfaits non plus mais sont un peu plus efficaces pour transmettre le message. Pour en savoir plus sur les alternatives au *barplot*, la conférence d’Eric Mauvière pour le réseau des *data scientists* de la statistique publique, dont le message principal est *“Désempilez vos figures”*, mérite le détour ([disponible sur le site ssphub.netlify.app/](https://ssphub.netlify.app/talk/2024-02-29-mauviere/)).

# 4. La même figure avec `Plotnine`

`plotnine` est le nouveau venu dans l’écosystème de la visualisation en `Python`. Cette librairie est développée par `Posit`, l’entreprise à l’origine de l’éditeur `RStudio` et de l’écosystème du *tidyverse* si central dans le langage `R`. Cette librairie vise à importer la logique de `ggplot` en `Python`, c’est-à-dire une grammaire des graphiques normalisée, lisible et flexible héritée de Wilkinson (2012).

<figure>
<img src="https://minio.lab.sspcloud.fr/lgaliana/generative-art/pythonds/elmo.jpg" alt="L’état d’esprit des habitués de ggplot2 quand ils découvrent plotnine" />
<figcaption aria-hidden="true">L’état d’esprit des habitués de <code>ggplot2</code> quand ils découvrent <code>plotnine</code></figcaption>
</figure>

Dans cette approche, un graphique est vu comme une succession de couches qui, une fois superposées, donneront la figure suivante. En soi, ce principe n’est pas différent de celui de `matplotlib`. Néanmoins, la grammaire utilisée par `plotnine` est beaucoup plus intuitive et normalisée, ce qui offrira beaucoup plus d’autonomie pour modifier sa figure.

<figure>
<img src="https://psyteachr.github.io/data-skills-v2/images/corsi/layers.png" alt="La logique de ggplot (et plotnine) par Lisa (2021), image elle-même empruntée à Field (2012)" />
<figcaption aria-hidden="true">La logique de <code>ggplot</code> (et <code>plotnine</code>) par <span class="citation" data-cites="Lisa_psyTeachR_Book_Template_2021">Lisa (2021)</span>, image elle-même empruntée à <span class="citation" data-cites="field2012discovering">Field (2012)</span></figcaption>
</figure>

Avec `plotnine`, il n’y a plus de point d’entrée dual figure-axe. Comme l’illustrent les slides ci-dessous :

1.  On initialise une figure
2.  On met à jour les couches (*layers*), un niveau d’abstraction très général concernant aussi bien les données représentées que les échelles des axes ou la couleur
3.  À la fin, on peut jouer sur l’esthétique en modifiant les labels des axes, de la légende, les titres, etc.

# 5. Premières agrégations temporelles

On va maintenant se concentrer sur la dimension temporelle de notre jeu de données à travers deux approches :

-   Un diagramme en barre synthétisant l’information de notre jeu de données de manière mensuelle ;
-   Des séries instructives sur la dynamique temporelle. Cela sera l’objet de la prochaine partie.

Avant cela, nous allons enrichir ces données pour bénéficier d’un historique plus long, permettant notamment d’avoir la période Covid dans nos données, ce qui présente un intérêt du fait de la dynamique particulière du trafic dans cette période (arrêt brutal, reprise très forte…).

In [40]:
import requests
import zipfile
import io
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd

list_useful_columns = [
        "Identifiant du compteur", "Nom du compteur",
        "Identifiant du site de comptage",
        "Nom du site de comptage",
        "Comptage horaire",
        "Date et heure de comptage"
    ]


# GENERIC FUNCTION TO RETRIEVE DATA -------------------------


def download_unzip_and_read(url, extract_to='.', list_useful_columns=list_useful_columns):
    """
    Downloads a zip file from the specified URL, extracts its contents, and reads the CSV file based on the filename pattern in the URL.

    Parameters:
    - url (str): The URL of the zip file to download.
    - extract_to (str): The directory where the contents of the zip file should be extracted.

    Returns:
    - df (DataFrame): The loaded pandas DataFrame from the extracted CSV file.
    """
    try:
        # Extract the file pattern from the URL (filename without the extension)
        file_pattern = url.split('/')[-1].replace('_zip/', '')


        # Send a GET request to the specified URL to download the file
        response = requests.get(url)
        response.raise_for_status()  # Ensure we get a successful response

        # Create a ZipFile object from the downloaded bytes
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Extract all the contents to the specified directory
            z.extractall(path=extract_to)
            print(f"Extracted all files to {os.path.abspath(extract_to)}")

        dir_extract_to = Path(extract_to)
        #dir_extract_to = Path(f"./{file_pattern}/")

        # Look for the file matching the pattern
        csv_filename = [
            f.name for f in dir_extract_to.iterdir() if f.suffix == '.csv'
        ]

        if not csv_filename:
            print(f"No file matching pattern '{file_pattern}' found.")
            return None

        # Read the first matching CSV file into a pandas DataFrame
        csv_path = os.path.join(dir_extract_to.name, csv_filename[0])
        print(f"Reading file: {csv_path}")
        df = pd.read_csv(csv_path, sep=";")

        # CONVERT TO GEOPANDAS
        df[['latitude', 'longitude']] = df['Coordonnées géographiques'].str.split(',', expand=True)
        df['latitude'] = pd.to_numeric(df['latitude'])
        df['longitude'] = pd.to_numeric(df['longitude'])
        gdf = gpd.GeoDataFrame(
            df, geometry=gpd.points_from_xy(df.longitude, df.latitude)
        )

        # CONVERT TO TIMESTAMP
        df["Date et heure de comptage"] = (
            df["Date et heure de comptage"]
            .astype(str)
            .str.replace(r'\+.*', '', regex=True)
        )
        df["Date et heure de comptage"] = pd.to_datetime(
            df["Date et heure de comptage"],
            format="%Y-%m-%dT%H:%M:%S",
            errors="coerce"
        )
        gdf = df.loc[
            :, list_useful_columns
        ]
        return gdf

    except requests.exceptions.RequestException as e:
        print(f"Error: The downloaded file has not been found: {e}")
        return None
    except zipfile.BadZipFile as e:
        print(f"Error: The downloaded file is not a valid zip file: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def read_historical_bike_data(year):
    dataset = "comptage_velo_donnees_compteurs"
    url_comptage = f"https://opendata.paris.fr/api/datasets/1.0/comptage-velo-historique-donnees-compteurs/attachments/{year}_{dataset}_csv_zip/"
    df_comptage = download_unzip_and_read(
        url_comptage, extract_to=f'./extracted_files_{year}'
    )
    if (df_comptage is None):
        url_comptage_alternative = url_comptage.replace("_csv_zip", "_zip")
        df_comptage = download_unzip_and_read(url_comptage_alternative, extract_to=f'./extracted_files_{year}')
    return df_comptage


# IMPORT HISTORICAL DATA -----------------------------

historical_bike_data = pd.concat(
    [read_historical_bike_data(year) for year in range(2018, 2024)]
)

rename_columns_dict = {
    "Identifiant du compteur": "id_compteur",
    "Nom du compteur": "nom_compteur",
    "Identifiant du site de comptage": "id",
    "Nom du site de comptage": "nom_site",
    "Comptage horaire": "sum_counts",
    "Date et heure de comptage": "date"
}


historical_bike_data = historical_bike_data.rename(
    columns=rename_columns_dict
)


# IMPORT LATEST MONTHS ----------------

import os
import requests
from tqdm import tqdm
import pandas as pd
import duckdb

url = 'https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptage-velo-donnees-compteurs/exports/parquet?lang=fr&timezone=Europe%2FParis'
filename = 'comptage_velo_donnees_compteurs.parquet'


# DOWNLOAD FILE --------------------------------

# Perform the HTTP request and stream the download
response = requests.get(url, stream=True)

if not os.path.exists(filename):
    # Perform the HTTP request and stream the download
    response = requests.get(url, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Get the total size of the file from the headers
        total_size = int(response.headers.get('content-length', 0))

        # Open the file in write-binary mode and use tqdm to show progress
        with open(filename, 'wb') as file, tqdm(
                desc=filename,
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
        ) as bar:
            # Write the file in chunks
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:  # filter out keep-alive chunks
                    file.write(chunk)
                    bar.update(len(chunk))
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
else:
    print(f"The file '{filename}' already exists.")


# READ FILE AND CONVERT TO PANDAS
query = """
SELECT id_compteur, nom_compteur, id, sum_counts, date
FROM read_parquet('comptage_velo_donnees_compteurs.parquet')
"""

# READ WITH DUCKDB AND CONVERT TO PANDAS
df = duckdb.sql(query).df()

df.head(3)


# PUT THEM TOGETHER ----------------------------

historical_bike_data['date'] = (
    historical_bike_data['date']
    .dt.tz_localize(None)
)

df["date"] = df["date"].dt.tz_localize(None)

historical_bike_data = (
    historical_bike_data
    .loc[historical_bike_data["date"] < df["date"].min()]
)

df = pd.concat(
    [historical_bike_data, df]
)

Pour commencer, reproduisons la troisième figure qui est, encore une fois, un *barplot*. Ici, sur le plan sémiologique, ce n’est pas justifié d’utiliser un *barplot*, une simple série suffirait à fournir une information similaire.

La première question du prochain exercice implique une première rencontre avec une donnée temporelle à travers une opération assez classique en séries temporelles : changer le format d’une date pour pouvoir faire une agrégation à un pas de temps plus large.

La figure avec les données depuis début 2022 aura cet aspect si elle a été construite avec `plotnine` :

Avec `seaborn`, elle ressemblera plutôt à ceci :

Si vous préférez représenter cela sous forme de *lollipop*[1]:

Enfin, sur l’ensemble de la période, la série prendra plutôt cette forme :

# 6. Première série temporelle

Il est plus commun de représenter les données ayant une dimension temporelle sous la forme de série que de barres empilées.

:

# 7. Des graphiques réactifs grâce aux librairies `Javascript`

## 7.1 L’écosystème disponible depuis `Python`

Les figures figées construites avec `matplotlib` ou `plotnine` sont figées et présentent ainsi l’inconvénient de ne pas permettre d’interaction avec le lecteur. Toute l’information doit donc être contenue dans la figure, ce qui peut la rendre difficile à lire. Si la figure est bien faite, avec différents niveaux d’information, cela peut bien fonctionner.

Il est néanmoins plus simple, grâce aux technologies *web*, de proposer des visualisations à plusieurs niveaux. Un premier niveau d’information, celui du coup d’œil, peut suffire à assimiler les principaux messages de la visualisation. Ensuite, un comportement plus volontaire de recherche d’information secondaire peut permettre d’en savoir plus. Les visualisations réactives, qui sont maintenant la norme dans le monde de la *dataviz*, permettent ce type d’approche : le lecteur d’une visualisation peut passer sa souris à la recherche d’informations complémentaires (par exemple, les valeurs exactes) ou cliquer pour faire apparaître des informations complémentaires sur la visualisation ou autour.

Ces visualisations reposent sur le même triptyque que l’ensemble de l’écosystème *web* : `HTML`, `CSS` et `JavaScript`. Les utilisateurs de `Python` ne vont jamais manipuler directement ces langages, qui demandent une certaine expertise, mais vont utiliser des librairies au niveau de `R` qui génèreront automatiquement tout le code `HTML`, `CSS` et `JavaScript` permettant de créer la figure.

Il existe plusieurs écosystèmes `Javascript` mis à disposition des développeurs.euses par le biais de `Python`. Les deux principales librairies sont [`Plotly`](https://plotly.com/python/), associée à l’écosystème `Javascript` du même nom, et [`Altair`](https://altair-viz.github.io/), associée à l’écosystème `Vega` et `Altair` en `Javascript`[2]. Pour permettre aux pythonistes de découvrir la librairie `Javascript` émergente [`Observable Plot`](https://observablehq.com/plot/), l’ingénieur de recherche français Julien Barnier a développé [`pyobsplot`](https://juba.github.io/pyobsplot/) une librairie `Python` permettant d’utiliser cet écosystème depuis `Python`.

L’interactivité ne doit pas juste être un gadget n’apportant pas de lisibilité supplémentaire, voire la détériorant. Il est rare de pouvoir se contenter de la figure produite sans avoir à fournir un travail supplémentaire pour la rendre efficace.

### 7.1.1 La librairie `Plotly`

Le package `Plotly` est une surcouche à la librairie `Javascript` `Plotly.js` qui permet de créer et manipuler des objets graphiques de manière très flexible afin de produire des objets réactifs sans avoir à recourir à Javascript.

Le point d’entrée recommandé est le module `plotly.express` ([documentation ici](https://plotly.com/python/plotly-express/)) qui offre une approche intuitive pour construire des graphiques pouvant être modifiés *a posteriori* si besoin (par exemple pour *customiser* les axes).

## 7.2 Réplication de l’exemple précédent avec `Plotly`

Les modules suivants seront nécessaires pour construire des graphiques avec `plotly`:

[1] I removed the color on the y-axis as I find it adds little to the figure and may even degrade the clarity of the message.

[2] The names of these libraries are inspired by the Summer Triangle constellation, of which Vega and Altair are two members.

In [56]:
import plotly
import plotly.express as px

## 7.3 La librairie `altair`

Pour cet exemple, nous allons reconstruire notre figure précédente.

Comme `ggplot`/`plotnine`, `Vega` est un écosystème graphique visant à proposer une implémentation de la grammaire des graphiques de Wilkinson (2012). La syntaxe de `Vega` est donc basée sur un principe déclaratif : on déclare une construction par couches et transformations de données progressives.

À l’origine, `Vega` est basée sur une syntaxe JSON, d’où son lien fort avec `Javascript`. Néanmoins, il existe une API Python qui permet de faire ce type de figures interactives nativement en Python. Pour comprendre la logique de construction d’un code `altair`, voici comment répliquer la figure précédente avec :

In [68]:
import altair as alt

color_scale = alt.Scale(domain=[True, False], range=['green', 'red'])

fig2 = (
    alt.Chart(df1)
    .mark_bar()
    .encode(
        x=alt.X('average(sum_counts):Q', title='Moyenne du comptage par heure sur la période sélectionnée'),
        y=alt.Y('nom_compteur:N', sort='-x', title=''),
        color=alt.Color('top:N', scale=color_scale, legend=alt.Legend(title="Top")),
        tooltip=[
            alt.Tooltip('nom_compteur:N', title='Nom du compteur'),
            alt.Tooltip('sum_counts:Q', title='Moyenne horaire')
            ]
    ).properties(
        title='Les 10 compteurs avec la moyenne horaire la plus élevée'
    ).configure_view(
        strokeOpacity=0
    )
)

fig2.interactive()

Field, A. 2012. « Discovering Statistics Using R ». Sage.

Lisa, DeBruine. 2021. « psyTeachR Book Template ». <https://github.com/psyteachr/template/>.

Wilkinson, Leland. 2012. *The grammar of graphics*. Springer.